# Pattern Matching
## Deterministic Finite Automaton

### High-Level Description (Conceptual)

**Goal**: Efficiently find all occurrences of a pattern within a text using a **deterministic finite automaton (DFA)**.

**Approach**:

1. Preprocess the pattern to build a **transition table** that encodes how the automaton moves from state to state on each symbol.
2. Each state represents how much of the pattern has been matched so far (i.e., prefix matched).
3. Simulate the automaton over the input text:

   * Follow transitions based on each character in the text.
   * A match is found each time the final state is reached.

---

### Low-Level Description (Implementation)

**`__init__`**:

* Initializes the DFA:

  * `num_states` = pattern length + 1 (including initial state).
  * Builds the `transition_table` where each key is `(state, symbol)` and value is the next state.

**`_build_transition_table`**:

* Constructs the transition table by calling `_calculate_state` for every state-symbol pair.

**`_calculate_state`**:

* Computes the next DFA state based on the current state and input symbol.
* Logic:

  * Append the symbol to the current pattern prefix.
  * Return the length of the longest suffix of this new string that matches a prefix of the pattern.
  * Implements a concept similar to the **KMP prefix function**.

**`process_input`**:

* Simulates the DFA over the input sequence:

  * Returns a list of states visited while processing the string.

**`find_occurrences`**:

* Uses `process_input` to find all positions where the automaton reaches the final state.
* A match is reported when the final DFA state (equal to `len(pattern)`) is reached.
* Returns total matches and their starting positions.

**`visualize`**:

* Prints the transition table for debugging and visualization.

**`get_next_state`**:

* Provides safe access to transitions (raises an error if symbol not in alphabet).

**`pattern_overlap`** (helper function):

* Returns the length of the longest suffix of one string that matches a prefix of another.
* Used for state calculation in `_calculate_state`.



In [23]:
class AutomatosFinitos :

    def __init__(self, alphabet: set, pattern: str):
        """
        Initializes the finite automaton for pattern matching.

        Args:
            alphabet (set): Set of all possible characters (input symbols).
            pattern (str): The pattern to search for in texts.
        """
        if not pattern:
            raise ValueError ('Pattern cannot be empty')
        if not alphabet:
            raise ValueError ('Alphabet cannot be empty')
        
        self.num_states = len(pattern) + 1  # One state for each prefix + initial
        self.alphabet = alphabet
        self.pattern = pattern
        self.transition_table = {}          # (state, symbol) --> next_state
        self._build_transition_table()      # Precompute transitions

    def _build_transition_table(self):

        """
        Constructs the transition table for the DFA by calculating
        the next state from each state on each symbol in the alphabet.
        """

        for state in range(self.num_states):
            for symbol in self.alphabet:
                next_state = self._calculate_state(state, symbol)
                self.transition_table[(state, symbol)] = next_state

    def _calculate_state(self, current_state: int, symbol: str):
        """
        Given a current state and an input symbol, calculates the next state
        based on the longest suffix of the accumulated string that matches a prefix
        of the pattern (i.e., KMP-style prefix function logic).

        Returns:
            int: The index of the next state.
        """

        # Immediate optimization for initial transition
        if current_state == 0 and symbol == self.pattern[0]:
            return 1
        
        # Build candidate by appending symbol to current pattern prefix
        candidate = self.pattern[:current_state] +  symbol
        max_overlap = min(current_state + 1, len(self.pattern))

         # Try longest possible suffix of candidate that matches prefix of pattern
        for length in range(max_overlap, 0, -1):
            if candidate.endswith(self.pattern[:length]):
                return length
        return 0
    
    def process_input(self, sequence: str) :
        """
        Simulates DFA on the given input sequence.

        Args:
            sequence (str): The input string to be scanned.

        Returns:
            List[int]: List of states visited during simulation.
        """
        
        current_state = 0
        states_log = [current_state]

        # Transition through each symbol in input
        for char in sequence:
            current_state = self.transition_table.get(
                (current_state, char), 0
            )
            states_log.append(current_state)
        return states_log
    
    def find_occurrences (self, text: str):
        """
        Finds all starting positions of the pattern in the given text.

        Args:
            text (str): Text to search within.

        Returns:
            Tuple[int, List[int]]: Total number of matches and their positions.
        """

        states = self.process_input(text)
        pattern_length = len(self.pattern)

        # A match is found whenever we reach the final state
        matches = [
            index - pattern_length 
            for index, state in enumerate(states)
            if state == len (self.pattern)
        ]
        return len (matches), matches
    
    def visualize (self):
        """
        Prints a visual representation of the DFA transitions.
        """
        print(f"States: {self.num_states}")
        print(f"Search Pattern: {self.pattern}")
        print("Transition Map:")
        for state in sorted(set(s for s, _ in self.transition_table)):
            symbols = [k[1] for k in self.transition_table if k[0] == state]
            for symbol in sorted(symbols):
                dest = self.transition_table[(state, symbol)]
                print(f"  ({state}, {symbol}) → {dest}")

    def get_next_state(self, current_state: int, symbol: str):
        """ Safe wrapper for transition logic.

        Args:
            current_state (int): The current DFA state.
            symbol (str): The symbol to process.

        Returns:
            int: Next state after consuming the symbol.

        Raises:
            ValueError: If the symbol is not part of the alphabet."""
        
        if symbol not in self.alphabet:
            raise ValueError(f"Symbol '{symbol}' not in alphabet")
        return self.transition_table.get((current_state, symbol), 0)
    
def pattern_overlap(suffix_candidate: str, prefix_source: str):
    """
    Helper function to compute the length of the longest suffix of one string
    that matches a prefix of another. Used in automaton construction.

    Args:
        suffix_candidate (str): The string to take the suffix from.
        prefix_source (str): The string to match prefix against.

    Returns:
        int: Length of the maximum overlap.
    """
    max_length = min(len(suffix_candidate), len(prefix_source))
    for length in range(max_length, 0, -1):
        if suffix_candidate[-length:] == prefix_source[:length]:
            return length
    return 0

In [25]:
alphabet = {'a', 'b','c'}
pattern = "abc"

# Create the finite automaton
automaton = AutomatosFinitos(alphabet, pattern)

# Visualize the automaton's transition table
print("Visualizing the automaton:")
automaton.visualize()

# Define a text to search for the pattern
text = "abcabcababc"

# Find occurrences of the pattern in the text
count, positions = automaton.find_occurrences(text)

# Output results
print("\n" + "="*25 + "Test" + "="*25 + "\n")
print(f"Text: {text}")
print(f"Pattern: {pattern}")
print(f"Number of occurrences: {count}")
print(f"Starting positions of occurrences: {positions}")

# Process the input sequence and show states visited
states_visited = automaton.process_input(text)
print(f"\nStates visited during processing: {states_visited}")

Visualizing the automaton:
States: 4
Search Pattern: abc
Transition Map:
  (0, a) → 1
  (0, b) → 0
  (0, c) → 0
  (1, a) → 1
  (1, b) → 2
  (1, c) → 0
  (2, a) → 1
  (2, b) → 0
  (2, c) → 3
  (3, a) → 1
  (3, b) → 0
  (3, c) → 0

=========================Test=========================

Text: abcabcababc
Pattern: abc
Number of occurrences: 3
Starting positions of occurrences: [0, 3, 8]

States visited during processing: [0, 1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 3]


## Burrows-Wheeler Transform

### High-Level Description (Conceptual)

**Goal**: Efficient pattern matching and data compression using the **Burrows-Wheeler Transform (BWT)** and associated structures.

**Approach**:

1. **Transform** the original sequence using the BWT:

   * Generate all cyclic rotations of the sequence.
   * Sort the rotations lexicographically.
   * The BWT string is the last column of this sorted matrix.
2. **Build supporting structures**:

   * **Suffix array** (optional): stores the original starting index of each suffix in sorted order.
   * **Last-to-First mapping (LF-mapping)**: links positions in the BWT's last column to the first column.
3. **Backward search**:

   * Scan the pattern from right to left.
   * Use LF-mapping to narrow the matching range in the BWT.
4. Optionally, **recover** the original string using the BWT and a reconstruction loop.

---

### Low-Level Description (Implementation)

**`build_bwt`**:

* Generates all cyclic rotations of the input.
* Sorts them lexicographically.
* Returns the last character of each sorted rotation as the BWT string.

**`build_suffix_array`**:

* Constructs and sorts all suffixes of the original string.
* Stores their starting positions.

**`last_to_first`**:

* Builds a mapping from positions in the last column of the BWT to their corresponding positions in the first (sorted) column.
* Uses `find_ith_occurrence` to resolve character occurrence alignment.

**`bw_matching`**:

* Performs **backward pattern matching**:

  * Initializes top and bottom bounds in the BWT.
  * Iteratively narrows the range based on each character from the pattern.
  * Uses LF-mapping to move from the last to the first column.

**`bw_matching_pos`**:

* Uses `bw_matching` to find matching row indices.
* Maps those to original positions via the suffix array.
* Returns the sorted list of start positions of the pattern in the original text.

**`bwt_reverse`**:

* Reverses the BWT to recover the original sequence.
* Builds the BWT matrix row by row by prepending characters and sorting.
* Returns the row that ends with the terminal symbol (`$`), stripped of that symbol.


In [21]:
from typing import List

def find_ith_occurrence(lst: List[str], char: str, i: int) -> int:
    """
    Finds the index of the i-th occurrence of a character in a list.

    Args:
        lst (List[str]): The list to search in.
        char (str): The character to search for.
        i (int): The occurrence number (1-based).

    Returns:
        int: The index of the i-th occurrence.

    Raises:
        ValueError: If the character does not occur i times.
    """
    count = 0
    for idx, c in enumerate(lst):
        if c == char:
            count += 1
            if count == i:
                return idx
    raise ValueError(f"{char} does not occur {i} times.")

class BWT:
    """
    A class to perform Burrows-Wheeler Transform (BWT) operations, including:
    - BWT construction
    - Suffix Array generation
    - Pattern matching using the BWT
    """

    def __init__(self, seq: str, buildsufarray: bool = False):
        """
        Initializes the BWT class with the given sequence.

        Args:
            seq (str): The input sequence.
            buildsufarray (bool): If True, builds the suffix array.
        """
        self.seq = seq + "$" if "$" not in seq else seq       # Ensure the sequence ends with a terminal symbol $
        self.bwt = self.build_bwt(self.seq)                    # Build BWT from the sequence
        self.sa = self.build_suffix_array(self.seq) if buildsufarray else [] # Optionally build suffix array

    def build_bwt(self, text: str) -> str:
        """
        Constructs the Burrows-Wheeler Transform of the sequence.

        Args:
            text (str): The input string with a terminal character.

        Returns:
            str: The BWT string.
        """
        rotations = [text[i:] + text[:i] for i in range(len(text))]  # Generate all cyclic rotations
        rotations.sort()                                             # Sort rotations lexicographically
        return ''.join(rot[-1] for rot in rotations)                 # Return last characters of each rotation

    def build_suffix_array(self, text: str) -> List[int]:
        """
        Constructs the suffix array of the sequence.

        Args:
            text (str): The input string.

        Returns:
            List[int]: The suffix array as a list of starting positions.
        """
        suffixes = [(text[i:], i) for i in range(len(text))]
        suffixes.sort()                                     # Sort suffixes lexicographically
        return [suffix[1] for suffix in suffixes]

    def last_to_first(self) -> List[int]:
        """
        Builds the Last-to-First mapping used in BWT matching.

        Returns:
            List[int]: A list mapping each index in the BWT (last column)
                       to its corresponding index in the first column.
        """
        first_col = sorted(self.bwt)        # First column is sorted version of BWT
        last_to_first_mapping = []

        # Keep track of occurrences of each character
        count = {}
        for i, char in enumerate(self.bwt):
            count[char] = count.get(char, 0) + 1
            occurrence_index = count[char]
            pos_in_first_col = find_ith_occurrence(first_col, char, occurrence_index)
            last_to_first_mapping.append(pos_in_first_col)

        return last_to_first_mapping

    def bw_matching(self, pattern: str) -> List[int]:
        """
        Performs backward search using BWT to find the pattern.

        Args:
            pattern (str): The pattern to search for.

        Returns:
            List[int]: A list of BWT matrix row indices where the pattern occurs.
        """
        lf_mapping = self.last_to_first()
        top = 0
        bottom = len(self.bwt) - 1

        # Process pattern from right to left
        while top <= bottom and pattern:
            symbol = pattern[-1]        # Last character of pattern
            pattern = pattern[:-1]

            sub_bwt = self.bwt[top:bottom + 1]      # Subrange of BWT

            if symbol in sub_bwt:
                # Find new top/bottom range for symbol
                top_index = sub_bwt.index(symbol) + top
                bottom_index = bottom - sub_bwt[::-1].index(symbol)

                # Move to corresponding rows in first column via LF mapping
                top = lf_mapping[top_index]
                bottom = lf_mapping[bottom_index]
            else:
                return [] # Symbol not found, pattern does not exist

        # Return all matching row indices in the final narrowed range
        return list(range(top, bottom + 1))

    def bw_matching_pos(self, pattern: str) -> List[int]:
        """
        Returns the original positions where the pattern occurs,
        using the suffix array and BWT matching.

        Args:
            pattern (str): The pattern to find.

        Returns:
            List[int]: Sorted list of starting positions of the pattern.
        """
        matches = self.bw_matching(pattern)

        if not matches:
            return []

        # Map matching rows to positions using suffix array
        positions = [self.sa[m] for m in matches]
        positions.sort()
        return positions

def bwt_reverse(bwt: str) -> str:
    """
    Reverses the Burrows-Wheeler Transform and recovers the original sequence.

    Args:
        bwt (str): The BWT-transformed string.

    Returns:
        str: The original sequence (without the terminal character).
    """
    n = len(bwt)
    table = [''] * n

    # Reconstruct rows by prepending characters and sorting
    for _ in range(n):
        table = sorted([bwt[i] + table[i] for i in range(n)])
    
    # Find the row that ends with '$' (original string)
    for row in table:
        if row.endswith('$'):
            return row.rstrip('$')
    return ""


#teste
if __name__ == "__main__":
    seq = "BANANA"
    bwt_instance = BWT(seq, buildsufarray=True)

    res = seq, bwt_instance.bwt
    suffix = bwt_instance.sa

    pattern = "ANA"
    positions = bwt_instance.bw_matching_pos(pattern)
    padrao = pattern, positions

    print("\n" + "="*25 + "Test" + "="*25 + "\n")
    print("Original e BWT: ", res)                       
    print("Suffix Array: ", suffix)                   
    print("Search pattern and positions: ", padrao)                   

    original_recuperado = bwt_reverse(bwt_instance.bwt)
    print("Recovered sequence:", original_recuperado) 



=========================Test=========================

Original e BWT:  ('BANANA', 'ANNB$AA')
Suffix Array:  [6, 5, 3, 1, 0, 4, 2]
Search pattern and positions:  ('ANA', [1, 3])
Recovered sequence: BANANA


## Tries and Suffix Trees)
### High-Level Description (Conceptual)

**Goal**: Efficiently search for exact pattern matches (or prefixes) within a collection of strings or a single string using tree-based data structures.

---

### I. Trie (Prefix Tree)

**Use Case**: Efficient matching of **multiple patterns** within a single text, especially useful when patterns are known in advance.

**Approach**:

1. Each node represents a **character**.
2. Insert patterns character by character into a tree rooted at node 0.
3. To match:

   * Traverse from the root following characters from the input text.
   * A match is found when a **leaf node** (no children) is reached.

---

### Low-Level Description (Trie)

**`add_node`**:

* Adds a new node and connects it via a character from the parent node.

**`add_pattern`**:

* Inserts a single pattern into the Trie.
* If a character is not already a child of the current node, a new node is created.

**`trie_from_patterns`**:

* Inserts multiple patterns into the Trie using `add_pattern`.

**`prefix_trie_match`**:

* Checks if any pattern matches a **prefix** of the input text.
* Returns the matched prefix if found.

**`trie_matches`**:

* Scans the entire text and returns **all occurrences** of patterns stored in the Trie.

---

### II. Suffix Tree (Naive Implementation)

**Use Case**: Efficient substring queries within a **single large string**. Ideal for **finding repeated substrings**, **pattern matching**, or **text indexing**.

**Approach**:

1. Build a **tree of all suffixes** of the input text.
2. Each path from the root to a leaf represents a suffix.
3. To search:

   * Traverse edges character by character for the input pattern.
   * If traversal is successful, all leaves below represent **occurrence positions**.

---

### Low-Level Description (Suffix Tree)

**`_add_node`**:

* Adds a new node to the tree under a given parent node with a character label.

**`_add_suffix`**:

* Adds a suffix to the tree, one character at a time.

**`build`**:

* Appends a `$` to the input text to ensure uniqueness.
* Inserts all suffixes of the modified text into the tree.

**`_get_leaves_below`**:

* Recursively collects all leaf indices under a node—these are **starting positions** of matching substrings.

**`find_pattern`**:

* Attempts to match a pattern by traversing the tree.
* If successful, returns all positions (leaf indices) where the pattern occurs.

**`print_tree`**:

* Displays the suffix tree structure for debugging.

---

### Summary: Trie vs. Suffix Tree

| Feature         | Trie                        | Suffix Tree                         |
| --------------- | --------------------------- | ----------------------------------- |
| Purpose         | Match many patterns in text | Match substrings in one string      |
| Build Time      | O(N × M) (N = #patterns)    | O(N²) (naive, for text length N)    |
| Search Time     | O(M) per search             | O(M) per search                     |
| Space           | Can be high (many patterns) | High, but scalable with compression |
| Pattern Sources | External patterns           | Substrings of internal text         |

In [2]:
class Trie:
    def __init__(self) -> None:
        """
        Initializes an empty Trie with a root node (node 0).
        """
        self.nodes = {0: {}}  # root node
        self.num = 0          # node counter

    def print_trie(self) -> None:
        """
        Prints the structure of the Trie for debugging.
        """
        for k in self.nodes.keys():
            print(k, "->", self.nodes[k])

    def add_node(self, origin: int, symbol: str) -> None:
        """
        Adds a new node to the Trie.

        :param origin: ID of the node where the edge starts
        :param symbol: character for the edge label
        """
        self.num += 1  # Increment node ID counter
        self.nodes[origin][symbol] = self.num   # Add new edge from origin
        self.nodes[self.num] = {}    # Create an empty dictionary for the new node

    def add_pattern(self, p: str) -> None:
        """
        Inserts a single pattern into the Trie.

        :param p: pattern to insert
        """
        pos = 0      # Position in the pattern
        node = 0     # Start from the root node
        while pos < len(p):
             # If current symbol is not a child of current node, add a new node
            if p[pos] not in self.nodes[node]:
                self.add_node(node, p[pos])
            # Move to the next node in the path
            node = self.nodes[node][p[pos]]
            pos += 1

    def trie_from_patterns(self, pats: list[str]) -> None:
        """
        Inserts a list of patterns into the Trie.

        :param pats: list of patterns
        """
        for p in pats:
            self.add_pattern(p)

    def prefix_trie_match(self, text: str) -> str | None:
        """
        Searches for the longest prefix in the Trie that matches the beginning of text.

        :param text: input string to match
        :return: matched prefix string or None
        """
        pos = 0     # Current position in text
        match = ""  # Accumulated matched prefix
        node = 0    # Start from the root node
        while pos < len(text):
            char = text[pos]
            if char in self.nodes[node]:
                node = self.nodes[node][char]
                match += char
                # If node has no children, it represents the end of a pattern
                if self.nodes[node] == {}:  # terminal node
                    return match
                pos += 1
            else:
                 # Mismatch occurred
                return None
        return None # End of text reached without completing a pattern

    def trie_matches(self, text: str) -> list[tuple[int, str]]:
        """
        Searches for all patterns in the Trie that match substrings in text.

        :param text: the input string
        :return: list of tuples (start_position, matched_string)
        """
        res = []
        for i in range(len(text)):
            # Try matching from every position in the text
            match = self.prefix_trie_match(text[i:])
            if match is not None:
                res.append((i, match))
        return res


In [22]:
patterns = ["GAT", "CCT", "GAG"]
t = Trie()
t.trie_from_patterns(patterns)
t.print_trie()
print("\n" + "="*25 + "Test" + "="*25 + "\n")
print("Match único:", t.prefix_trie_match("GAGATCCTA"))
print("Todos os matches:", t.trie_matches("GAGATCCTA"))

0 -> {'G': 1, 'C': 4}
1 -> {'A': 2}
2 -> {'T': 3, 'G': 7}
3 -> {}
4 -> {'C': 5}
5 -> {'T': 6}
6 -> {}
7 -> {}

=========================Test=========================

Match único: GAG
Todos os matches: [(0, 'GAG'), (2, 'GAT'), (5, 'CCT')]


In [12]:
print("="*25 + "Test" + "="*25 + "\n")
patterns = ["AGAGAT", "AGC", "AGTCC", "CAGAT", "CCTA", "GAGAT", "GAT", "TC", "GA"]
t = Trie()
t.trie_from_patterns(patterns)
print("Match único:", t.prefix_trie_match("GAGATCCTA"))
print("Todos os matches:", t.trie_matches("GAGATCCTA"))

=========================Test=========================

Match único: GAGAT
Todos os matches: [(0, 'GAGAT'), (2, 'GAT'), (4, 'TC'), (5, 'CCTA')]


In [5]:
class SuffixTree:
    def __init__(self):
        """
        Initializes the suffix tree with a root node.
        Each node is represented as a tuple: (leaf_index, children_dict).
            - leaf_index: starting index of the suffix if the node is a leaf; otherwise -1
            - children_dict: dictionary mapping characters to child node IDs
        """
        self.nodes = {0: (-1, {})}  # node_id: (leaf_index or -1, {symbol: child_node_id})
        self.node_count = 0         # Counter for assigning unique node IDs
        self.text = ""              # Text that the suffix tree is built on

    def _add_node(self, parent, symbol, leaf_index=-1):
        """
        Adds a new node as a child of the given parent node.

        :param parent: index of the parent node
        :param symbol: character representing the edge to the new node
        :param leaf_index: suffix index if it's a leaf node, otherwise -1
        """
        self.node_count += 1      
         # Add the new node to the parent's children                         
        self.nodes[parent][1][symbol] = self.node_count
         # Initialize the new node with leaf_index and empty childre
        self.nodes[self.node_count] = (leaf_index, {})      

    def _add_suffix(self, suffix, index):
        """
        Adds a suffix to the tree.

        :param suffix: suffix string to be inserted
        :param index: starting index of the suffix in the original text
        """
        node = 0 # Start from the root node
        for i, char in enumerate(suffix):
            # If there's no edge with this character, create a new node
            if char not in self.nodes[node][1]:
                is_leaf = (i == len(suffix) - 1)  # If it's the last character of the suffix
                self._add_node(node, char, index if is_leaf else -1)
            # Move to the child node
            node = self.nodes[node][1][char]

    def build(self, text):
        """
        Builds the suffix tree from the given input string.

        :param text: input string to build the suffix tree from
        """
        self.text = text + "$"          # Append a terminal character to ensure unique suffixes
        for i in range(len(self.text)):
             # Add all suffixes starting at position i
            self._add_suffix(self.text[i:], i)

    def _get_leaves_below(self, node_id):
        """
        Recursively collects all leaf indices under the given node.

        :param node_id: ID of the starting node
        :return: list of leaf indices
        """
        leaf_index, children = self.nodes[node_id]
        if leaf_index >= 0:
            return [leaf_index]      # Base case: current node is a leaf

        results = []
        for child in children.values():
            results.extend(self._get_leaves_below(child))        # Recursive call for each child
        return results

    def find_pattern(self, pattern):
        """
        Searches for a pattern in the suffix tree.

        :param pattern: string pattern to search for
        :return: list of starting positions in the text where the pattern occurs, or None
        """
        node = 0        # Start from the root
        for char in pattern:
            if char in self.nodes[node][1]:
                node = self.nodes[node][1][char]
            else:
                return None     # Pattern not found
        return self._get_leaves_below(node)

    def print_tree(self):
        """
        Prints the structure of the suffix tree.
        Shows node IDs, leaf indices, and child transitions.
        """
        for node_id, (leaf_index, children) in self.nodes.items():
            if leaf_index >= 0:
                print(f"{node_id} : leaf index {leaf_index}")
            else:
                print(f"{node_id} -> children: {children}")


In [11]:
if __name__ == "__main__":
    st = SuffixTree()
    st.build("banana")
    st.print_tree()

    print("\n" + "="*25 + "Test" + "="*25 + "\n")
    print("Matches for 'ana':", st.find_pattern("ana"))
    print("Matches for 'na':", st.find_pattern("na"))
    print("Matches for 'a':", st.find_pattern("a"))
    print("Matches for 'x':", st.find_pattern("x"))


0 -> children: {'b': 1, 'a': 8, 'n': 14, '$': 22}
1 -> children: {'a': 2}
2 -> children: {'n': 3}
3 -> children: {'a': 4}
4 -> children: {'n': 5}
5 -> children: {'a': 6}
6 -> children: {'$': 7}
7 : leaf index 0
8 -> children: {'n': 9, '$': 21}
9 -> children: {'a': 10}
10 -> children: {'n': 11, '$': 19}
11 -> children: {'a': 12}
12 -> children: {'$': 13}
13 : leaf index 1
14 -> children: {'a': 15}
15 -> children: {'n': 16, '$': 20}
16 -> children: {'a': 17}
17 -> children: {'$': 18}
18 : leaf index 2
19 : leaf index 3
20 : leaf index 4
21 : leaf index 5
22 : leaf index 6

=========================Test=========================

Matches for 'ana': [1, 3]
Matches for 'na': [2, 4]
Matches for 'a': [1, 3, 5]
Matches for 'x': None
